## Serving a Stable Diffusion Model with Ray Serve
This template shows you how to develop and test the model locally and deploy it into production.

### Step 1: Install python dependencies

In [ ]:
!pip install -q diffusers==0.25.0 transformers==4.36.2 accelerate==0.25.0

### Step 2: Deploy the model locally
- Open a new terminal (ctrl+shift+` in VS Code.)
- Run the command below to deploy your model at http://localhost:8000.  

```bash
# Run the following in a new VSCode terminal. It's needed to run this in a separate
# terminal instead of in this notebook since `serve run` is a blocking command.
$ serve run app:entrypoint
```

### Step 3: Send test requests to the running model
- An image should be generated in the current directory

In [ ]:
import requests
endpoint = "http://localhost:8000/imagine"

def generate_image(prompt, image_size):
    req = {"prompt": prompt, "img_size": image_size}
    resp = requests.get(endpoint, params=req)
    return resp.content

image = generate_image("twin peaks sf in basquiat painting style", 640)
filename = "image.png"
with open(filename, "wb") as f:
    f.write(image)

from IPython.display import Image
Image(filename=filename) 

## Next steps

That's it for deploying a model for testing in workspaces. To learn how to deploy this model to prod, go to the `publish-to-prod.ipynb` notebook.